# User Input

## How much initial capital we assume and for how many weeks to run the program . Defining higher number of weeks will require more computation time

In [1]:
we = 52
cap = 7000

# Importing required Libraries

In [2]:
import pandas as pd
from pulp import *
from num2words import *

# Data Preprocessing

In [3]:
inventory0 = pd.read_csv('inventory.csv')
sales0 = pd.read_csv('Weekly-Sales.csv')
productdesc0 = pd.read_csv('products.csv').set_index('Product_ID')
stores0 = pd.read_csv('stores.csv')

In [4]:
# Keeping More weeks will require more computation time
week = [None]*(we)
for i in range(1,(we+1)):
    week[i-1] =i

In [5]:
stores = [None]*50
for i in range(0,50):
    stores[i] = i+1
products = [None]*35
for i in range(0,35):
    products[i]=i+1

In [6]:
sales = {}
for s in stores:
    for p in products:
        for w in week:
            sales[(s,p,w)]=(sales0[(sales0['Store ID']==s)&(sales0['Product ID']==p)&(sales0['Week']==w)&(sales0['Year of Date']==2017)]['Count of sales'])
            if len(sales[s,p,w]>0):
                sales[s,p,w]=int(sales[s,p,w])
            else:
                sales[s,p,w]=0
del sales0     

In [7]:
openinginventory = {}
for s in stores:
    for p in products:
        openinginventory[(s,p)]=inventory0[(inventory0['Store_ID']==s)&(inventory0['Product_ID']==p)]['Stock_On_Hand']
        if len(openinginventory[(s,p)]>0):
            openinginventory[(s,p)] = int(openinginventory[(s,p)])
        else:
            openinginventory[(s,p)] = 0
del inventory0

In [8]:
Pcost = {}
Pprice = {}
Pprofit = {}
for p in products:
    Pcost[p]=float(productdesc0.loc[p]['Product_Cost'][1:5])
    Pprice[p]=float(productdesc0.loc[p]['Product_Price'][1:5])
    Pprofit[p] = Pprice[p] - Pcost[p]

# Model

## After computation of results of every week , the result will be stored here in these dictionaries 

In [9]:
l = {} # Goods Sold
t = {} # capital unused
k = {} # Goods Bought
u = {} # Inventory after buying weeks

## Model for Week 1

In [10]:
# Defining our objective variable
z = LpProblem('Maximum-Profit',LpMaximize)
# Defining Decision variable
x = LpVariable.dict('Buying',(stores,products,week),lowBound=0,cat='Integer')
y = LpVariable.dict('Selling',(stores,products,week),lowBound=0,cat='Integer')
capital_available = LpVariable.dict('Capital_Available',(week),lowBound=0,cat='Continous')
capital_unused = LpVariable.dict('Capital_Unused',(week),lowBound=0,cat='Continous')
inventory = LpVariable.dict('Inventory',(stores,products,week),lowBound=0,cat='Integer')
# Objective Function
z += lpSum(y[s,p,1]*Pprofit[p] for s in stores for p in products)
# Constrains
## We cannot spend more than available capital
z += lpSum(x[s,p,1]*Pcost[p] for s in stores for p in products) + capital_unused[1] <= cap
z += capital_unused[1] == cap - lpSum(x[s,p,1]*Pcost[p] for s in stores for p in products)
for s in stores:
    for p in products:
        ## Goods bought are added to the inventory
        z += inventory[s,p,1] == x[s,p,1]
        ## We cannot sell more than demanded
        z += y[s,p,1] <= sales[s,p,1]
        ## We cannot sell more than we have in our inventory
        z += y[s,p,1] <= inventory[s,p,1]
status = z.solve()
print("Solution is :",LpStatus[status])
for s in stores:
    for p in products:
        # Resolving the issue of over buying
        if (x[s,p,1].varValue>sales[s,p,1]):
            capital_unused[1].varValue = capital_unused[1].varValue + lpSum((x[s,p,1].varValue-sales[s,p,w])*Pcost[p])
            x[s,p,1].varValue=sales[s,p,1]
        # Products not selected are none so setting them to zero
        if y[s,p,w].varValue is None:
            y[s,p,w].varValue = 0
        if x[s,p,w].varValue is None:
            x[s,p,w].varValue = 0
        if inventory[s,p,w].varValue is None:
            inventory[s,p,w].varValue = 0
        # Passing the decision variables to dictionary 
        l[s,p,1]=y[s,p,1].varValue
        k[s,p,1]=x[s,p,1].varValue
        u[s,p,1]=inventory[s,p,1].varValue
t[1] = capital_unused[1].varValue
# Printing result
print("For week 1 profit is :",lpSum(l[s,p,1]*Pprofit[p] for s in stores for p in products))

Solution is : Optimal
For week 1 profit is : 8321.71


## Model for Remaining weeks

In [11]:
# Iteratively solving for every week
for w in range(2,we+1):
    # Defining our objective variable
    z = LpProblem('Maximum-Profit',LpMaximize)
    # Decision Variables
    x = LpVariable.dict('Buying',(stores,products,week),lowBound=0,cat='Integer')
    y = LpVariable.dict('Selling',(stores,products,week),lowBound=0,cat='Integer')
    capital_available = LpVariable.dict('Capital_Available',(week),lowBound=0,cat='Continous')
    capital_unused = LpVariable.dict('Capital_Unused',(week),lowBound=0,cat='Continous')
    inventory = LpVariable.dict('Inventory',(stores,products,week),lowBound=0,cat='Integer')
    # Objective Function
    z += lpSum(y[s,p,w]*Pprofit[p] for s in stores for p in products)
    # Capital of this a is revenue of previous week and capital unused of previous week
    z += capital_available[w] == lpSum(l[s,p,w-1]*Pprice[p] for s in stores for p in products) + t[w-1]
    # We cannot buy goods worth more than capital we have
    z += lpSum(x[s,p,w]*Pcost[p] for s in stores for p in products) + capital_unused[w] <= capital_available[w]
    z += capital_unused[w] == capital_available[w] - lpSum(x[s,p,w]*Pcost[p] for s in stores for p in products)
    for s in stores: 
        for p in products:
           # Adds up to the inventory
           z += inventory[s,p,w] == x[s,p,w]
           # We cannot sell more than demanded
           z += y[s,p,w] <= sales[s,p,w]
           # Cannot sell more than inventory
           z += y[s,p,w] <= inventory[s,p,w]
           ## We cannot purchase more than demanded
           #z += x[s,p,w] <= sales[s,p,w]
    status = z.solve()       
    print("Solution is :",LpStatus[status])
    # Passing the decision variables to dictionary
    for s in stores:
        for p in products:
            # Resolving the issue of over buying
            if (x[s,p,w].varValue>sales[s,p,w]):
                capital_unused[w].varValue = capital_unused[w].varValue + lpSum((x[s,p,w].varValue-sales[s,p,w])*Pcost[p])
                x[s,p,w].varValue=sales[s,p,w]
            # Products not selected are none so setting them to zero
            if y[s,p,w].varValue is None:
                y[s,p,w].varValue = 0
            if x[s,p,w].varValue is None:
                x[s,p,w].varValue = 0
            if inventory[s,p,w].varValue is None:
                inventory[s,p,w].varValue = 0
            # Passing the decision variables to dictionary 
            l[s,p,w]=y[s,p,w].varValue
            k[s,p,w]=x[s,p,w].varValue
            u[s,p,w]=inventory[s,p,w].varValue
    t[w] = capital_unused[w].varValue
    # Printing result
    print("For week",w,"profit is :",lpSum(y[s,p,w].varValue*Pprofit[p] for s in stores for p in products))


Solution is : Optimal
For week 2 profit is : 14300.19
Solution is : Optimal
For week 3 profit is : 21481.269999999986
Solution is : Optimal
For week 4 profit is : 26338.269999999982
Solution is : Optimal
For week 5 profit is : 29255.309999999987
Solution is : Optimal
For week 6 profit is : 29689.66999999997
Solution is : Optimal
For week 7 profit is : 30622.779999999988
Solution is : Optimal
For week 8 profit is : 29254.26
Solution is : Optimal
For week 9 profit is : 33808.729999999974
Solution is : Optimal
For week 10 profit is : 28932.97999999999
Solution is : Optimal
For week 11 profit is : 31875.299999999992
Solution is : Optimal
For week 12 profit is : 28365.629999999983
Solution is : Optimal
For week 13 profit is : 28307.469999999994
Solution is : Optimal
For week 14 profit is : 31062.759999999987
Solution is : Optimal
For week 15 profit is : 35941.57
Solution is : Optimal
For week 16 profit is : 28250.519999999986
Solution is : Optimal
For week 17 profit is : 31671.80999999998
S

In [13]:
del x
del y
del inventory
del capital_available
del capital_unused

# Analyzing our Result

In [14]:
profit = round(value(lpSum(l[s,p,w]*Pprofit[p] for s in stores for p in products for w in range(1,we))))
print("Profit of",we,"weeks is :",profit)
print("Profit in Words :",(num2words(profit,lang='en_IN')))

Profit of 52 weeks is : 1590132
Profit in Words : fifteen lakh, ninety thousand, one hundred and thirty-two


In [15]:
max_profit = round(value(lpSum(sales[s,p,w]*Pprofit[p] for s in stores for p in products for w in week)))
print("Maximum possible profit in",we,"weeks is :",max_profit)
print("Max possible Profit in Words :",(num2words(max_profit,lang='en_IN')))

Maximum possible profit in 52 weeks is : 1675908
Max possible Profit in Words : sixteen lakh, seventy-five thousand, nine hundred and eight


In [16]:
for w in range(1,2):
    # Capital of Week depends on previous week revenue and unused capital
    print("\nCapital Available in week",w,":",cap)
    # Capital required in week is Demand * Cost
    print("Capital Required in week",w,":",lpSum(sales[s,p,w]*Pcost[p] for s in stores for p in products))
    # Capital used is Buyed * cost
    print("Capital Used in week",w,":",lpSum(k[s,p,w]*Pcost[p] for s in stores for p in products))
    # Capital not used
    print("Capital Not Used in week",w,":",t[w])
    # Sold * Price
    print("Revenue of week for sales",w,":",lpSum(l[s,p,w]*Pprice[p] for s in stores for p in products))
    # Sold * Profit
    print("Profit of week",w,":",lpSum(l[s,p,w]*Pprofit[p] for s in stores for p in products))

for w in range(2,we+1) :
    # Capital of Week depends on previous week revenue and unused capital
    print("\nCapital Available in week",w,":",value(lpSum(l[s,p,w-1]*Pprice[p] for s in stores for p in products)) + t[w-1])
    # Capital required in week is Demand * Cos
    print("Capital Required in week",w,":",lpSum(sales[s,p,w]*Pcost[p] for s in stores for p in products))
    # Capital used is Buyed * cost
    print("Capital Used in week",w,":",lpSum(k[s,p,w]*Pcost[p] for s in stores for p in products))
    # Capital not used
    print("Capital Not Used in week",w,":",t[w])
    # Sold * Price
    print("Revenue of week for sales",w,":",lpSum(l[s,p,w]*Pprice[p] for s in stores for p in products))
    # Sold * Profit
    print("Profit of week",w,":",lpSum(l[s,p,w]*Pprofit[p] for s in stores for p in products))
    


Capital Available in week 1 : 7000
Capital Required in week 1 : 73571.75
Capital Used in week 1 : 6999.18
Capital Not Used in week 1 : 0.82
Revenue of week for sales 1 : 15320.890000000001
Profit of week 1 : 8321.71

Capital Available in week 2 : 15321.710000000001
Capital Required in week 2 : 61901.439999999966
Capital Used in week 2 : 15321.48
Capital Not Used in week 2 : 0.23
Revenue of week for sales 2 : 29621.66999999999
Profit of week 2 : 14300.19

Capital Available in week 3 : 29621.89999999999
Capital Required in week 3 : 66469.01000000005
Capital Used in week 3 : 29621.85
Capital Not Used in week 3 : 0.05
Revenue of week for sales 3 : 51103.12000000003
Profit of week 3 : 21481.269999999986

Capital Available in week 4 : 51103.170000000035
Capital Required in week 4 : 69357.11999999997
Capital Used in week 4 : 51101.05999999994
Capital Not Used in week 4 : 2.11
Revenue of week for sales 4 : 77439.32999999999
Profit of week 4 : 26338.269999999982

Capital Available in week 5 : 

# Key Performance Indicators

## Inventory turnover ratio

In [17]:
cost_of_goods_sold = lpSum(l[s,p,w]*Pcost[p] for s in stores for p in products for w in week)
avg_inventory = lpSum(l[s,p,w]*Pcost[p] for s in stores for p in products for w in week)*(1/we)
inventory_turnover_ratio = cost_of_goods_sold/avg_inventory
print("Inventory was replaced",inventory_turnover_ratio,"times in",we,"weeks")

Inventory was replaced 51.99999999999999 times in 52 weeks


## Revenue per unit

In [18]:
total_revenue = lpSum(l[s,p,w]*Pprice[p] for s in stores for p in products for w in week)
units = lpSum(l[s,p,w] for s in stores for p in products for w in week)
revenue_per_unit = total_revenue/units
print("Revenue per unit is",revenue_per_unit)

Revenue per unit is 13.923758890118343


## Gross Margin Return on Investment
The gross margin return on investment (GMROI) is an inventory profitability evaluation ratio that analyzes a firm's ability to turn inventory into cash above the cost of the inventory.

In [20]:
storage_cost = value(total_revenue)*0.05
gross_margin_roi = profit/storage_cost
print("We earned",gross_margin_roi,"times for every dollar of investment")

We earned 5.640423461885956 times for every dollar of investment


## Order Fill rate

In [21]:
total_orders = lpSum(sales[s,p,w] for s in stores for p in products for w in week)
order_fill_rate = (units/total_orders)*100
print("We were able to fullfill",order_fill_rate,"of orders")

We were able to fullfill 96.72845404165871 of orders


# Conclusion

We required initially a capital of $75000 but had only $7000 . We concentrated on how to use use those $7000 in week-1 so that we can have more capital in next week . This process was repeated for 52 weeks